In [59]:
from script.scraper_functions import *
from script.soupifier import *
from serde.json import to_json
import numpy as np
from itertools import chain
from yarl import URL

**TODO**:

- `ruff` $\Rightarrow$ linter, à utiliser plus dans le cadre du dvpt
- `pytest` avec subprocess.run pour tester le code
- `pytest-coverage`
- `poetry` avec gestion des dépendances
- `black` pour automatiquement indenter, etc.
- `doctest` pour tester les exemples des docstrings
- `bandit` pour gérer les vulnérabilités

***

- `yarl` pour travailler avec des URL

## Le code ci-dessous lance le `scraper`

- Tu peux changer le nombre d'*URL* de **19** à 1 ou 2 pour tester ça prendra beaucoup moins longtemps à générer
- Lien [`Darty`](https://www.darty.com/nav/extra/list?seller=311289-0-327183-311290-311291-328771-311452&cat=89054&state=new_offer&persist_sold_by_darty=true) $\Rightarrow$ en choisissant les produits uniquement vendus par Darty + Neufs on a 567 produits.
- Avoir un comparatif Boulanger/Darty pourrait être intéressant
- Voir pour récupérer les images des téléphones pour plus tard dans l'appli - voir : https://discuss.streamlit.io/t/add-image-and-header-to-streamlit-dataframe-table/36065/3

In [60]:
URL()

URL('')

In [61]:
session = create_session()
url_list = create_url_list(URL, 1)
pages = read_pages(url_list=url_list, session=session)

In [ ]:
all_urls = extract_all_urls(pages)
print("--------------------")
product_pages = extract_all_pages(all_urls, session)

Elapsed time for extract_all_urls function: 0.488 seconds.

--------------------
Product 1 -> Successfull Extraction.
Product 2 -> Successfull Extraction.
Product 3 -> Successfull Extraction.
Product 4 -> Successfull Extraction.
Product 5 -> Successfull Extraction.
Product 6 -> Successfull Extraction.
Product 7 -> Successfull Extraction.
Product 8 -> Successfull Extraction.
Product 9 -> Successfull Extraction.
Product 10 -> Successfull Extraction.
Product 11 -> Successfull Extraction.
Product 12 -> Successfull Extraction.
Product 13 -> Successfull Extraction.
Product 14 -> Successfull Extraction.
Product 15 -> Successfull Extraction.
Product 16 -> Successfull Extraction.
Product 17 -> Successfull Extraction.
Product 18 -> Successfull Extraction.
Product 19 -> Successfull Extraction.
Product 20 -> Successfull Extraction.
Product 21 -> Successfull Extraction.
Product 22 -> Successfull Extraction.
Product 23 -> Successfull Extraction.
Product 24 -> Successfull Extraction.
Product 25 -> Successfull Extraction.
Product 26 -> Successfull Extraction.


Elapsed time for extract_all_pages function: 59.688 seconds.

Il faudrait créer une fonction pour la génération de la liste des smartphones + l'export en `json`

In [ ]:
smartphone_list = list()
for article in product_pages:
    soup = BeautifulSoup(article, "html.parser")
    dict_features = extract_features(soup)
    smartphone = smartphone_characteristics(dict_features)
    smartphone_list.append(smartphone)

In [ ]:
smartphones_json = to_json(smartphone_list)

In [ ]:
file_path = "data/smartphones.json"

with open(file_path, "w", encoding="utf-8") as json_file:
    json_file.write(smartphones_json)

## Charge le Dataframe dans `polars`

In [1]:
import polars as pl
import polars.selectors as cs
from script.polars_gazprom import *
from itertools import chain

In [2]:
df_brut = pl.read_json("data/smartphones.json")

### Pipeline

Sur `535` extractions, il y a `487` téléphones valides.

In [3]:
df = NordStream(df_brut)

In [4]:
len(df.columns) - 3

35

In [8]:
from pathlib import Path

root = Path(".").resolve()
data_folder = root / "data"
path_file = data_folder / "df_clean.csv"

In [6]:
df.write_csv(path_file, separator=";")

In [24]:
import plotly.express as px
from plotly.subplots import make_subplots

In [75]:
median = df.select(pl.col("price").median()).item()
mean = df.select(pl.col("price").mean()).item()
std = df.select(pl.col("price").std()).item()
pearson_skewness_coeff = (3*(mean-median))/ std

In [76]:
pearson_skewness_coeff

0.6722932133095527

Le résultat indique que l'asymétrie de la distribution est positive => indique une distribution décalée à gauche de la médiane

In [94]:
# The probability density function is nonnegative everywhere, and the area under the entire curve = 1.
hist = px.histogram(
    x="price",
    opacity=0.8,
    data_frame=df,
    histnorm="probability density",
    title="Distribution des prix - fonction de densité de probabilité",
)
hist.update_layout(
    xaxis=dict(ticksuffix=" €"),
    paper_bgcolor="#222222",
    plot_bgcolor="#222222",
    font_color="white",
    margin=dict(l=30, r=30, b=30, pad=3),
)
hist.update_yaxes(title="")
hist.update_xaxes(title="")

In [ ]:
df.select(cs.by_dtype(pl.NUMERIC_DTYPES)).corr()

# Matrice de corrélation

In [3]:
import statsmodels.formula.api as smf
import statsmodels.formula as sm
from stargazer.stargazer import Stargazer
import numpy as np
from dataclasses import dataclass
from script.hedonic_regression import *

In [12]:
y_1 = "price"
y_2 = "logprice"
X = [
    "screen_type",
    "das_limbs",
    "das_head",
    "upgrade_storage",
    "screen_size",
    "fast_charging",
    "repairability_index",
    "made_in",
    "ram",
    "storage",
    "brand",
    "induction",
]

In [10]:
f_1 = formula(y_1, X)
f_2 = formula(y_2, X)
res_1 = execute_ols(df, f_1)
res_2 = execute_ols(df, f_2)

On ajoute le prix prédit par la régression à notre dataframe

In [12]:
df = df.with_columns(pl.from_numpy(res_1.predict(), schema=["predicted_price"]))

In [ ]:
Stargazer([res_1, res_2])

In [3]:
sfa_file = data_folder / "sfa_results.csv"

In [8]:
sfa_df = pl.read_csv(sfa_file, ignore_errors=True)

In [ ]:
sfa_df.select("model", "price", "predicted_price", "efficiency").sort("efficiency")

In [ ]:
# res2 = res.get_robustcov_results(cov_type='HC1')
# robuste std errors